## --------------------------Testing Accuracy on Validation Dataset---------------------------- 

In [1]:
import nltk
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sb
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore")
import unidecode
#from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.stem import PorterStemmer
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import matplotlib.animation as animation
import operator
#import plotly.express as px
from collections import Counter
%matplotlib inline
import pandas_profiling

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [59]:
data=pd.read_csv("logistic_regression.csv")
#data=data.drop('tokenized',axis=1)
#data=data.drop('Unnamed: 0',axis=1)
data.head()

,Unnamed: 0,text,Tweet_tokenized,sentiment
0,0,last session day,"['last', 'session', 'day']",neutral
1,1,shanghai also realli excit precis skyscrap...,"['shanghai', 'also', 'realli', 'excit', 'preci...",neutral
2,2,recess hit veroniqu branquinho quit company ...,"['recess', 'hit', 'veroniqu', 'branquinho', 'q...",neutral
3,3,happi bday,"['happi', 'bday']",neutral
4,4,I like it,"['I', 'like', 'it']",neutral


In [60]:
data.shape

(3534, 4)

### Pre - Processing 

In [61]:
data.dropna
data.describe()

,Unnamed: 0
count,3534.000000
mean,1766.500000
std,1020.322253
min,0.000000
25%,883.250000
50%,1766.500000
75%,2649.750000
max,3533.000000


In [62]:
## Convert to lower case
data['text'] = data['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
data['text'].head()

0                                     last session day
1    shanghai also realli excit precis skyscrap gal...
2    recess hit veroniqu branquinho quit company shame
3                                           happi bday
4                                            i like it
Name: text, dtype: object

In [63]:
## Stopwards Removal
data['text'] = data['text'].apply(lambda x : ' '.join([word for word in str(x).split() if not word in set(stopwords.words('english'))]))

In [64]:
## Lemmatization
lemmatizer = WordNetLemmatizer()
data['text'] = data['text'].apply(lambda x : ' '.join([lemmatizer.lemmatize(word) for word in str(x).split()]))

In [65]:
from textblob import TextBlob
data['text'][:5].apply(lambda x: str(TextBlob(x).correct()))

0                                     last session day
1    shanghai also really exit precise skyscrap gal...
2    recess hit veroniqu branquinho quit company shame
3                                            happy day
4                                                 like
Name: text, dtype: object

In [66]:
def clean_tweets(tweet):
    tweet = re.sub('http://[A-Za-z0-9./]+','',tweet) 
    # remove special characters 
    tweet = re.sub('[^ a-zA-Z0-9]', '', tweet)
    # remove Numbers
    tweet = re.sub('[0-9]', '', tweet)
    return tweet
data['text'] = data['text'].apply(clean_tweets)

In [9]:
def tokenize(text):
    return word_tokenize(text)
data['tokenized'] = data['text'].apply(tokenize)
data.head()

,Unnamed: 0,text,sentiment,tokenized
0,0,last session day httptwitpiccomezh,negative,"[last, session, day, httptwitpiccomezh]"
1,1,shanghai also realli excit precis skyscrap gal...,neutral,"[shanghai, also, realli, excit, precis, skyscr..."
2,2,recess hit veroniqu branquinho quit company shame,positive,"[recess, hit, veroniqu, branquinho, quit, comp..."
3,3,happi bday,neutral,"[happi, bday]"
4,4,httptwitpiccomwp like,positive,"[httptwitpiccomwp, like]"


### Feature Extraction using TF-IDF Vectorizer 

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create the transform
cv = TfidfVectorizer(max_features=5000) 
x = cv.fit_transform(data['text'].tolist()).toarray()

In [68]:
data.sentiment[data.sentiment == 'positive'] = 1
data.sentiment[data.sentiment == 'neutral'] = 0
data.sentiment[data.sentiment == 'negative'] = 2
y=data['sentiment'].astype('float')
print(y)

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
3529    1.0
3530    0.0
3531    0.0
3532    0.0
3533    0.0
Name: sentiment, Length: 3534, dtype: float64


### Modelling 

In [69]:
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import expon,uniform,randint

#Sklearn imports
from sklearn import linear_model
from sklearn.model_selection import train_test_split,RandomizedSearchCV,cross_val_score,cross_val_predict,validation_curve
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [70]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0) #Basic train_test_split works here
y_test.shape

(884,)

In [48]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')

In [49]:
svc.fit(x_train,y_train)

SVC(kernel='linear')

In [ ]:
y_pred=svc.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

### Random Forest Classifier 

In [74]:
from sklearn.ensemble import RandomForestClassifier
m = RandomForestClassifier(n_estimators=40, min_samples_leaf=3, n_jobs=-1)
m.fit(x_train, y_train)
predictions = m.predict(x_test)
print(classification_report(y_test, predictions))
print("Accuracy :",accuracy_score(y_test, predictions),'\n')

              precision    recall  f1-score   support

         0.0       0.84      0.99      0.91       710
         1.0       0.88      0.30      0.45        69
         2.0       0.72      0.17      0.28       105

    accuracy                           0.84       884
   macro avg       0.81      0.49      0.55       884
weighted avg       0.83      0.84      0.80       884

Accuracy : 0.8371040723981901 



### Logistic Regression 

In [72]:
# With single train test split
clf = LogisticRegression(solver='liblinear').fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8099547511312217
[[709   0   1]
 [ 66   1   2]
 [ 99   0   6]]
              precision    recall  f1-score   support

         0.0       0.81      1.00      0.90       710
         1.0       1.00      0.01      0.03        69
         2.0       0.67      0.06      0.11       105

    accuracy                           0.81       884
   macro avg       0.83      0.36      0.34       884
weighted avg       0.81      0.81      0.73       884



### Multinomial Naive Bayes 

In [73]:
## Multinomial NB with single train test split
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB().fit(x_train, y_train)

y_pred=nb_model.predict(x_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8031674208144797
[[710   0   0]
 [ 69   0   0]
 [105   0   0]]
              precision    recall  f1-score   support

         0.0       0.80      1.00      0.89       710
         1.0       0.00      0.00      0.00        69
         2.0       0.00      0.00      0.00       105

    accuracy                           0.80       884
   macro avg       0.27      0.33      0.30       884
weighted avg       0.65      0.80      0.72       884



In [2]:
data=pd.read_csv("results.csv")

In [3]:
data.head()

,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -...,positive
2,"Recession hit Veronique Branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive


In [4]:
data.dropna
data.describe()

,text,sentiment
count,3534,3534
unique,3530,3
top,#NAME?,neutral
freq,5,1430


In [5]:
## Convert to lower case
data['text'] = data['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
data['text'].head()

0     last session of the day http://twitpic.com/67ezh
1    shanghai is also really exciting (precisely --...
2    recession hit veronique branquinho, she has to...
3                                          happy bday!
4               http://twitpic.com/4w75p - i like it!!
Name: text, dtype: object

In [6]:
## Stopwards Removal
data['text'] = data['text'].apply(lambda x : ' '.join([word for word in str(x).split() if not word in set(stopwords.words('english'))]))

In [7]:
## Lemmatization
lemmatizer = WordNetLemmatizer()
data['text'] = data['text'].apply(lambda x : ' '.join([lemmatizer.lemmatize(word) for word in str(x).split()]))

In [8]:
from textblob import TextBlob
data['text'][:5].apply(lambda x: str(TextBlob(x).correct()))

0            last session day http://twitpic.com/67ezh
1    shanghai also really exciting (precisely -- sk...
2    recession hit veronique branquinho, quit compa...
3                                           happy day!
4                 http://twitpic.com/4w75p - like it!!
Name: text, dtype: object

In [9]:
def clean_tweets(tweet):
    tweet = re.sub('http://[A-Za-z0-9./]+','',tweet) 
    # remove special characters 
    tweet = re.sub('[^ a-zA-Z0-9]', '', tweet)
    # remove Numbers
    tweet = re.sub('[0-9]', '', tweet)
    return tweet
data['text'] = data['text'].apply(clean_tweets)

In [10]:
def tokenize(text):
    return word_tokenize(text)
data['tokenized'] = data['text'].apply(tokenize)
data.head()

,text,sentiment,tokenized
0,last session day,neutral,"[last, session, day]"
1,shanghai also really exciting precisely skysc...,positive,"[shanghai, also, really, exciting, precisely, ..."
2,recession hit veronique branquinho quit compan...,negative,"[recession, hit, veronique, branquinho, quit, ..."
3,happy bday,positive,"[happy, bday]"
4,like it,positive,"[like, it]"


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create the transform
cv = TfidfVectorizer(max_features=5000) 
x = cv.fit_transform(data['text'].tolist()).toarray()

In [12]:
data.sentiment[data.sentiment == 'positive'] = 1
data.sentiment[data.sentiment == 'neutral'] = 0
data.sentiment[data.sentiment == 'negative'] = 2
y=data['sentiment'].astype('float')
print(y)

0       0.0
1       1.0
2       2.0
3       1.0
4       1.0
       ... 
3529    2.0
3530    1.0
3531    2.0
3532    1.0
3533    1.0
Name: sentiment, Length: 3534, dtype: float64


In [13]:
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import expon,uniform,randint

#Sklearn imports
from sklearn import linear_model
from sklearn.model_selection import train_test_split,RandomizedSearchCV,cross_val_score,cross_val_predict,validation_curve
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [14]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0) #Basic train_test_split works here
y_test.shape

(884,)

In [15]:
from sklearn.ensemble import RandomForestClassifier
m = RandomForestClassifier(n_estimators=40, min_samples_leaf=3, n_jobs=-1)
m.fit(x_train, y_train)
predictions = m.predict(x_test)
print(classification_report(y_test, predictions))
print("Accuracy :",accuracy_score(y_test, predictions),'\n')

              precision    recall  f1-score   support

         0.0       0.55      0.75      0.64       355
         1.0       0.73      0.64      0.69       273
         2.0       0.68      0.43      0.52       256

    accuracy                           0.62       884
   macro avg       0.66      0.61      0.62       884
weighted avg       0.65      0.62      0.62       884

Accuracy : 0.6244343891402715 



In [16]:
# With single train test split
clf = LogisticRegression(solver='liblinear').fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.6357466063348416
[[286  35  34]
 [103 165   5]
 [128  17 111]]
              precision    recall  f1-score   support

         0.0       0.55      0.81      0.66       355
         1.0       0.76      0.60      0.67       273
         2.0       0.74      0.43      0.55       256

    accuracy                           0.64       884
   macro avg       0.68      0.61      0.63       884
weighted avg       0.67      0.64      0.63       884



In [17]:
## Multinomial NB with single train test split
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB().fit(x_train, y_train)

y_pred=nb_model.predict(x_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.5984162895927602
[[289  37  29]
 [112 159   2]
 [165  10  81]]
              precision    recall  f1-score   support

         0.0       0.51      0.81      0.63       355
         1.0       0.77      0.58      0.66       273
         2.0       0.72      0.32      0.44       256

    accuracy                           0.60       884
   macro avg       0.67      0.57      0.58       884
weighted avg       0.65      0.60      0.58       884



In [18]:
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import TimeDistributed

Using TensorFlow backend.


In [20]:
from sklearn.model_selection import train_test_split
text = data['text'].values
labels = data['sentiment'].values
text_train, text_test, y_train, y_test = train_test_split(text, labels)
print(text_train.shape, text_test.shape, y_train.shape, y_test.shape)

(2650,) (884,) (2650,) (884,)


In [21]:
vocab_size = 20000
maxlen = 500
embedding_size = 32

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(text)

X_train = tokenizer.texts_to_sequences(text_train)
x_test = tokenizer.texts_to_sequences(text_test)

X_train = pad_sequences(X_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [22]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=maxlen))
model.add(LSTM(128))
#model.add(Dense(64, activation='relu'))
#model.add(Dense(1, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

W0622 19:41:52.863784  6052 deprecation.py:506] From C:\Users\dell\Anaconda3\lib\site-packages\tensorflow\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0622 19:41:52.902654  6052 deprecation.py:506] From C:\Users\dell\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           640000    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               82432     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 722,561
Trainable params: 722,561
Non-trainable params: 0
_________________________________________________________________


In [23]:
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        min_delta=1e-4,
        patience=3,
        verbose=1
    ),
    ModelCheckpoint(
        filepath='weights.h5',
        monitor='val_accuracy', 
        mode='max', 
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    )
]

In [25]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=60, validation_data=(x_test, y_test), epochs=2, callbacks=callbacks)

Train on 2650 samples, validate on 884 samples
Epoch 1/2
2640/2650 [============================>.] - ETA: 0s - loss: 0.3873 - acc: 0.3064

W0622 19:46:50.421300  6052 callbacks.py:1259] Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,acc,val_loss,val_acc
W0622 19:46:50.421300  6052 callbacks.py:989] Can save best model only with val_accuracy available, skipping.


2650/2650 [==============================] - 35s 13ms/sample - loss: 0.3877 - acc: 0.3068 - val_loss: 0.2818 - val_acc: 0.3281
Epoch 2/2
2640/2650 [============================>.] - ETA: 0s - loss: 0.1724 - acc: 0.3095

W0622 19:47:21.650425  6052 callbacks.py:1259] Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,acc,val_loss,val_acc
W0622 19:47:21.650425  6052 callbacks.py:989] Can save best model only with val_accuracy available, skipping.


2650/2650 [==============================] - 31s 12ms/sample - loss: 0.1732 - acc: 0.3091 - val_loss: 0.2142 - val_acc: 0.3247
